<a href="https://colab.research.google.com/github/CuriousandTime/SportsBetting-Multiplatform-OddsSpread-StatisticalArbitrage/blob/main/Colab_AutomaticRobot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **体育博彩多平台赔率差统计套利**

####清除所有旧文件从新开始

In [ ]:
! rm -rf NBA-Machine-Learning-Sports-Betting
! rm -rf *

####导入依赖关系和获取API密钥
[点击到the-odds-api获取API_KEY](https://the-odds-api.com)

In [ ]:
!pip install XlsxWriter


In [ ]:
import requests
import xlsxwriter
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Border, Side, Font, Alignment, PatternFill, numbers
# 替换为自己的API_KEY
API_KEY = 'c9c0d0dc971f844e3dbe99a5b73c0d'


####定义常量和进行API调用
- ``BET_SIZE`` 是您愿意为每个赌注下的货币金额（美元）。

- 例如：如果你定义``BET_SIZE``为100，你就是告诉程序，你的投注金额是100美元平均到两家或者多家博彩平台。

In [ ]:
import requests

SPORT = 'upcoming'  # 从 /sports 端点中使用 sport_key，或使用 'upcoming' 来查看所有体育运动的下一场比赛
REGIONS = 'us'  # uk | us | eu | au。如果以逗号分隔，则可以指定多个区域。
MARKETS = 'h2h'  # h2h | spreads | totals。如果以逗号分隔，则可以指定多个市场。
ODDS_FORMAT = 'decimal'  # decimal | american
DATE_FORMAT = 'iso'  # iso | unix
BET_SIZE = 100

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': 'c9c0d0dc971f844e3dbe99a5b73c0d',  # 同样替换为自己的API_KEY
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
).json()


In [ ]:
# 打印赔率
print(odds_response)

### 活动类别
- 每个 ``Event``对象代表一个独立的体育事件
- 参数 ``data `` 包含从API调用中收到的所有赔率数据。

In [ ]:
BOOKMAKER_INDEX = 0  # bookmaker在best_odds中的下标
NAME_INDEX = 1  # name在best_odds中的下标
ODDS_INDEX = 2  # odds在best_odds中的下标
FIRST = 0  # 用于获取某个列表的第一个元素

class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']  # 体育项目key
        self.id = data['id']  # 事件id
        self.commence_time = data['commence_time']  # 比赛时间

    def find_best_odds(self):
        # 可能出现的结果的数量
        num_outcomes = len(self.data['bookmakers'][FIRST]['markets'][FIRST]['outcomes'])
        self.num_outcomes = num_outcomes

        # 找到每个事件中每个结果的最佳赔率
        best_odds = [[None, None, float('-inf')] for _ in range(num_outcomes)]
        # [Bookmaker, Name, Price]

        bookmakers = self.data['bookmakers']
        for index, bookmaker in enumerate(bookmakers):

            # 确定每个博彩公司提供的赔率
            for outcome in range(num_outcomes):

                # 确定是否有博彩公司的赔率优于当前的最佳赔率
                bookmaker_odds = float(bookmaker['markets'][FIRST]['outcomes'][outcome]['price'])  # 该bookmaker所提供的该outcome的赔率
                current_best_odds = best_odds[outcome][ODDS_INDEX]  # 当前最佳的该outcome的赔率

                if bookmaker_odds > current_best_odds:  # 如果该bookmaker的该outcome的赔率更优
                    best_odds[outcome][BOOKMAKER_INDEX] = bookmaker['title']  # 更新best_odds
                    best_odds[outcome][NAME_INDEX] = bookmaker['markets'][FIRST]['outcomes'][outcome]['name']  # 更新best_odds
                    best_odds[outcome][ODDS_INDEX] = bookmaker_odds  # 更新best_odds

        self.best_odds = best_odds
        return best_odds

    def arbitrage(self):
        total_arbitrage_percentage = 0
        for odds in self.best_odds:
            total_arbitrage_percentage += (1.0 / odds[ODDS_INDEX])

        self.total_arbitrage_percentage = total_arbitrage_percentage
        self.expected_earnings = (BET_SIZE / total_arbitrage_percentage) - BET_SIZE

        # 如果赔率的倒数之和小于1，就有机会进行套利。
        if total_arbitrage_percentage < 1:
            return True
        return False

    # 将decimal赔率转换为American赔率
    def convert_decimal_to_american(self):
        best_odds = self.best_odds
        for odds in best_odds:
            decimal = odds[ODDS_INDEX]  # 获取该outcome的decimal赔率
            if decimal >= 2:  # 如果是赔率 >= 2
                american = (decimal - 1) * 100  # 转换为american赔率
            elif decimal < 2:  # 如果是赔率 < 2
                american = -100 / (decimal - 1)  # 转换为american赔率
            odds[ODDS_INDEX] = round(american, 2)  # 更新best_odds中的
        return best_odds

    def calculate_arbitrage_bets(self):
        bet_amounts = []
        for outcome in range(self.num_outcomes):
            individual_arbitrage_percentage = 1 / self.best_odds[outcome][ODDS_INDEX] # 获取该outcome的套利百分比
            bet_amount = (BET_SIZE * individual_arbitrage_percentage) / self.total_arbitrage_percentage # 计算所需投注金额
            bet_amounts.append(round(bet_amount, 2)) # 将投注金额添加到列表中
        
        self.bet_amounts = bet_amounts # 将计算后的投注金额存入实例属性中
        return bet_amounts # 返回计算后的投注金额列表


### 解析事件和计算套利赌注
- `BET_SIZE` 是指你想在一个事件的结果中投注的美元金额。
- 这种计算将使用无偏套利，即无论结果如何，利润都是一样的。

In [ ]:
# 创建一个空列表用于存储解析后的事件
events = []

# 解析每个赔率响应中的数据并将其转换为Event对象
for data in odds_response:
    events.append(Event(data))
    print(data)
    print()

# 创建一个空列表用于存储套利事件
arbitrage_events = []

# 遍历每个事件并查找最佳赔率，如果找到套利机会则将其添加到arbitrage_events中
for event in events:
    best_odds = event.find_best_odds()
    if event.arbitrage():
        arbitrage_events.append(event)

# 遍历每个套利事件并计算套利赌注
for event in arbitrage_events:
    event.calculate_arbitrage_bets()
    event.convert_decimal_to_american()

### 创建数据框架并写入Excel文件

In [ ]:
# 计算最大结果数和套利机会数
MAX_OUTCOMES = max([event.num_outcomes for event in arbitrage_events])
ARBITRAGE_EVENTS_COUNT = len(arbitrage_events)

# 定义列名称
my_columns = ['编号', '赛事编号', '比赛时间', '预期收益'] + list(np.array([[f'博彩平台 #{outcome}', f'球队名称 #{outcome}', f'赔率 #{outcome}', f'购买金额 #{outcome}'] for outcome in range(1, MAX_OUTCOMES + 1)]).flatten())

# 创建空DataFrame
dataframe = pd.DataFrame(columns=my_columns)

In [ ]:
for event in arbitrage_events:
    # 打印(event.best_odds)
    row = []
    row.append(event.id)
    row.append(event.sport_key)
    row.append(event.commence_time) # 添加比赛时间
    row.append(round(event.expected_earnings, 2))
    for index, outcome in enumerate(event.best_odds):
        row.append(outcome[BOOKMAKER_INDEX])
        row.append(outcome[NAME_INDEX])
        row.append(outcome[ODDS_INDEX])
        row.append(event.bet_amounts[index])
    while len(row) < len(dataframe.columns):
        row.append('N/A')
    dataframe.loc[len(dataframe.index)] = row

In [ ]:
# 将结果保存为bets.xlsx文件
writer = pd.ExcelWriter('bets.xlsx')
dataframe.to_excel(writer, index=False)
writer.save()

### 格式化Excel文件

In [ ]:
BLACK = '000000'  # 黑色
LIGHT_GREY = 'D6D6D6'  # 浅灰色
DARK_GREY = '9F9F9F'  # 深灰色
RED = 'FEA0A0'  # 红色
BLUE = 'A0CEFE'  # 蓝色
GREEN = '22D49C'  # 蓝色
YELLOW = 'FFE540'  # 黄色

COLORS = [RED, BLUE]  # 颜色列表

ID_COLUMN_FILL = PatternFill(fill_type='solid', start_color=DARK_GREY, end_color=DARK_GREY)  # 单元格填充样式：ID 列
SPORT_KEY_COLUMN_FILL = PatternFill(fill_type='solid', start_color=LIGHT_GREY, end_color=LIGHT_GREY)  # 单元格填充样式：体育关键词列
COMMENCE_TIME_COLUMN_FILL = PatternFill(fill_type='solid', start_color=GREEN, end_color=GREEN)  # 单元格填充样式：比赛时间列
EXPECTED_EARNINGS_COLUMN_FILL = PatternFill(fill_type='solid', start_color=YELLOW, end_color=YELLOW)  # 单元格填充样式：预期收益列

CENTER_ALIGNMENT = Alignment(horizontal='center', vertical='bottom', indent=0)  # 单元格文本对齐方式：水平居中、垂直底部对齐

TOP_ROW_BORDER = Border(bottom=Side(border_style='thick', color=BLACK))  # 表头单元格边框
NORMAL_ROW_BORDER = Border(top=Side(border_style='thin', color=LIGHT_GREY), bottom=Side(border_style='thin', color=DARK_GREY))  # 普通单元格边框

 
wb = load_workbook('bets.xlsx')  # 加载 Excel 文件
ws = wb.active  # 获取活动工作表
ws.title = 'Upcoming'  # 将工作表名改为 Upcoming
# 修改列宽
for col in range(1, 26):
    ws.column_dimensions[chr(col + 64)].width = 20

            
    START_INDEX = chr(ord(START_INDEX) + 4)
# 针对 ID 列、体育关键词列和预期收益列，应用单元格填充和文本对齐样式
for cell in ws['A']:
    cell.fill = ID_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    
for cell in ws['B']:
    cell.fill = SPORT_KEY_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT

for cell in ws['C']:
    cell.fill = COMMENCE_TIME_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    
for cell in ws['D']:
    cell.fill = EXPECTED_EARNINGS_COLUMN_FILL
    cell.alignment = CENTER_ALIGNMENT
    cell.number_format = numbers.BUILTIN_FORMATS[7]  # 将单元格格式设为“货币”

START_INDEX = 'E'  # 第一列的字母索引为 E
for index in range(MAX_OUTCOMES):
    # 对于每个赔率结果，循环处理前三列
    for col in ws[START_INDEX : chr(ord(START_INDEX) + 3)]:
        for cell in col:
            # 设置单元格的填充和对齐方式
            color = COLORS[int(index % 2)]
            cell.fill = PatternFill(fill_type='solid', start_color=color, end_color=color)
            cell.alignment = CENTER_ALIGNMENT
            
            # 如果该单元格是第五列，则设置数字格式
            if cell.column in [8, 12, 16, 20]:
                cell.number_format = numbers.BUILTIN_FORMATS[7]  # 将单元格格式设为“货币”
          
    START_INDEX = chr(ord(START_INDEX) + 4)  # 将 START_INDEX 向右移动 4 个列

for cell in ws['1']:
    cell.border = TOP_ROW_BORDER

for row in range(2, ARBITRAGE_EVENTS_COUNT + 2):
    for cell in ws[str(row)]:
        cell.border = NORMAL_ROW_BORDER
    
wb.save('体育博彩跨平台套利计算结果.xlsx')  # 将结果保存文件为体育博彩跨平台套利计算结果.xlsx

In [ ]:
###结束所有操作，点击左边的文件夹下载体育博彩跨平台套利计算结果.xlsx文件即可。